In [1]:
from functions.MAE import *
from functions.benchmark import *
import torch
import numpy as np
import pandas as pd
import pickle
import time

In [ ]:
n_tasks = 1
combinations = pickle.load(open(f"evaluation_data/Test_combinations_{n_tasks}.pkl", "rb"))
test_data = pickle.load(open(f"evaluation_data/Test_data_{n_tasks}_100.pkl","rb"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
eval_loss_function= torch.nn.MSELoss
optimizer = torch.optim.RAdam
train_loss_function = torch.nn.SmoothL1Loss
cols = [str(i) for i in np.arange(0,300,1)]
dataset = create_dataset(combinations, cols, path="Completely_preprocessed_data")
# dataset = create_dataset(combinations, cols, path="C:\LocalData\Data\Completely_preprocessed_data")
n_epochs = 100

In [ ]:
dataloader = create_dataloader(dataset, 64, 25)
model = create_Vanilla_AE_separate(250, 300, combinations, 0, device)
s = time.time()
loss = train_model(model, n_epochs, dataloader, train_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [00:34<00:00,  2.95it/s]

0.992 33.805


In [ ]:
dataloader = create_dataloader(dataset, 64, 25)
model = create_TCN_AE_separate(300, combinations, 128, 32, 8, device, 8, 32, 6)
s = time.time()
loss = train_model(model, n_epochs, dataloader, train_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [00:49<00:00,  2.03it/s]


0.99 49.352


In [ ]:
dataloader = create_dataloader(dataset, 64, 25)
model = create_LSTM_AE_separate(100, 300, combinations, device)
s = time.time()
loss = train_model(model, n_epochs, dataloader, eval_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [06:18<00:00,  3.78s/it]


0.797 378.168


In [ ]:
train_data = torch.Tensor(dataset.df_torque.to_numpy()).unsqueeze(1).unsqueeze(1)
test_data_tensor = torch.Tensor(test_data.to_numpy()).unsqueeze(1).unsqueeze(1)
test_data_squeezed = test_data_tensor.squeeze()
model = create_DMOC(n_tasks, objective="hard-boundary", dimensions=128)
s = time.time()
net, loss, c_list = model.train(train_data, 100)
e = time.time()
t = e-s
eval_results = model.test(test_data_tensor)
fpr,tpr,thresholds = metrics.roc_curve(test_data_squeezed[:,-1].cpu(), eval_results)
result = metrics.auc(fpr,tpr)
print(round(result,3), round(t,3))

/home/snagelkerk/.local/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
100%|██████████| 4/4 [00:00<00:00, 141.58it/s]


  Epoch 1/100	 Time: 0.030	 Loss: 5.08355319


100%|██████████| 4/4 [00:00<00:00, 92.04it/s]


  Epoch 2/100	 Time: 0.045	 Loss: 2.25545442


100%|██████████| 4/4 [00:00<00:00, 221.05it/s]


...

  Epoch 98/100	 Time: 0.018	 Loss: 0.07854606


100%|██████████| 4/4 [00:00<00:00, 233.61it/s]


  Epoch 99/100	 Time: 0.018	 Loss: 0.07859138


100%|██████████| 4/4 [00:00<00:00, 234.82it/s]


  Epoch 100/100	 Time: 0.018	 Loss: 0.07074083
0.872 19.292


In [ ]:
n_tasks = 4
combinations = pickle.load(open(f"evaluation_data/Test_combinations_{n_tasks}.pkl", "rb"))
test_data = pickle.load(open(f"evaluation_data/Test_data_{n_tasks}_100.pkl","rb"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
eval_loss_function= torch.nn.MSELoss
optimizer = torch.optim.RAdam
train_loss_function = torch.nn.SmoothL1Loss
cols = [str(i) for i in np.arange(0,300,1)]
dataset = create_dataset(combinations, cols, path="Completely_preprocessed_data")
# dataset = create_dataset(combinations, cols, path="C:\LocalData\Data\Completely_preprocessed_data")
n_epochs = 100

In [ ]:
dataloader = create_dataloader(dataset, 64, 25)
model = create_Vanilla_MAE(250, 300, combinations, 0, device)
s = time.time()
loss = train_model(model, n_epochs, dataloader, train_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [02:15<00:00,  1.35s/it]

0.991 134.927


In [ ]:
dataloader = create_dataloader(dataset, 64, 25)
model = create_Vanilla_AE_separate(250, 300, combinations, 0, device)
s = time.time()
loss = train_model(model, n_epochs, dataloader, eval_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [02:11<00:00,  1.31s/it]

0.99 131.016


In [ ]:
dataloader = create_mixed_batch_dataloader(dataset, 64)
model = create_Vanilla_AE_shared(250, 300, combinations, 0, device)
s = time.time()
loss = train_model(model, n_epochs, dataloader, eval_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [02:38<00:00,  1.58s/it]

0.989 158.433


In [ ]:
dataloader = create_dataloader(dataset, 64, 25)
model = create_TCN_MAE(300, combinations, 128, 32, 8, 2, device, 8, 32, 6)
s = time.time()
loss = train_model(model, n_epochs, dataloader, train_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [03:08<00:00,  1.88s/it]


0.979 188.107


In [ ]:
dataloader = create_dataloader(dataset, 64, 25)
model = create_TCN_AE_separate(300, combinations, 128, 32, 8, device, 8, 32, 6)
s = time.time()
loss = train_model(model, n_epochs, dataloader, train_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [03:09<00:00,  1.89s/it]


0.978 188.767


In [ ]:
dataloader = create_mixed_batch_dataloader(dataset, 16)
model = create_TCN_AE_shared(300, 32, 8, 2, device, 8, 32, 6)
s = time.time()
loss = train_model(model, n_epochs, dataloader, train_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [04:04<00:00,  2.44s/it]


0.973 244.455


In [ ]:
dataloader = create_mixed_batch_dataloader(dataset, 16)
model = create_LSTM_MAE(200, 300, combinations, device)
s = time.time()
loss = train_model(model, n_epochs, dataloader, train_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [2:25:05<00:00, 87.06s/it] 


0.929 8705.781


In [ ]:
dataloader = create_dataloader(dataset, 64, 25)
model = create_GRU_AE_separate(250, 300, combinations, device)
s = time.time()
loss = train_model(model, n_epochs, dataloader, eval_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [29:06<00:00, 17.46s/it]


0.736 1746.293


In [ ]:
dataloader = create_mixed_batch_dataloader(dataset, 16)
model = create_LSTM_AE_shared(200, 300, device)
s = time.time()
loss = train_model(model, n_epochs, dataloader, eval_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [39:58<00:00, 23.98s/it]


0.884 2398.191


In [ ]:
train_data = torch.Tensor(dataset.df_torque.to_numpy()).unsqueeze(1).unsqueeze(1)
test_data_tensor = torch.Tensor(test_data.to_numpy()).unsqueeze(1).unsqueeze(1)
test_data_squeezed = test_data_tensor.squeeze()
model = create_DMOC(n_tasks, objective="soft-boundary", dimensions=32)
s = time.time()
net, loss, c_list = model.train(train_data, 100)
e = time.time()
t = e-s
eval_results = model.test(test_data_tensor)
fpr,tpr,thresholds = metrics.roc_curve(test_data_squeezed[:,-1].cpu(), eval_results)
result = metrics.auc(fpr,tpr)
print(round(result,3), round(t,3))

/home/snagelkerk/.local/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
100%|██████████| 4/4 [00:00<00:00, 73.75it/s]


  Epoch 1/100	 Time: 0.056	 Loss: 10368.31958008


100%|██████████| 4/4 [00:00<00:00, 19.16it/s]


  Epoch 2/100	 Time: 0.210	 Loss: 2148.34417725


100%|██████████| 4/4 [00:00<00:00, 60.91it/s]


...

  Epoch 98/100	 Time: 0.052	 Loss: 24.79498790


100%|██████████| 4/4 [00:00<00:00, 79.44it/s]


  Epoch 99/100	 Time: 0.052	 Loss: 24.55602565


100%|██████████| 4/4 [00:00<00:00, 79.51it/s]


  Epoch 100/100	 Time: 1.569	 Loss: 23.57446726
0.870 63.702


In [2]:
n_tasks = 16
combinations = pickle.load(open(f"evaluation_data/Test_combinations_{n_tasks}.pkl", "rb"))
test_data = pickle.load(open(f"evaluation_data/Test_data_{n_tasks}_100.pkl","rb"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
eval_loss_function= torch.nn.MSELoss
optimizer = torch.optim.RAdam
train_loss_function = torch.nn.SmoothL1Loss
cols = [str(i) for i in np.arange(0,300,1)]
dataset = create_dataset(combinations, cols, path="Completely_preprocessed_data")
# dataset = create_dataset(combinations, cols, path="C:\LocalData\Data\Completely_preprocessed_data")
n_epochs = 100

In [5]:
dataloader = create_dataloader(dataset, 64, 25)
model = create_Vanilla_MAE(250, 300, combinations, 0, device)
s = time.time()
loss = train_model(model, n_epochs, dataloader, train_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [9:10<00:00,  5.50s/it]


0.995 550.09


In [3]:
dataloader = create_dataloader(dataset, 64, 25)
model = create_Vanilla_AE_separate(250, 300, combinations, 0, device)
s = time.time()
loss = train_model(model, n_epochs, dataloader, eval_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True, lr_schedule = True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [9:12<00:00,  5.52s/it]


0.989 551.954


In [6]:
dataloader = create_mixed_batch_dataloader(dataset, 16)
model = create_Vanilla_AE_shared(100, 300, combinations, 0, device)
s = time.time()
loss = train_model(model, n_epochs, dataloader, eval_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [08:42<00:00,  5.22s/it]


0.987 522.39


In [8]:
dataloader = create_dataloader(dataset, 64, 25)
model = create_TCN_MAE(300, combinations, 128, 32, 8, 6, device, 8, 32, 6)
s = time.time()
loss = train_model(model, n_epochs, dataloader, train_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True, lr_schedule = True, schedule_milestones = [85]).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

  0%|          | 0/100 [00:00<?, ?it/s]/home/snagelkerk/.local/lib/python3.8/site-packages/torch/nn/modules/conv.py:309: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1003.)
  return F.conv1d(input, weight, bias, self.stride,
100%|██████████| 100/100 [13:43<00:00, 8.23s/it]


0.98 823.231


In [9]:
dataloader = create_dataloader(dataset, 64, 25)
model = create_TCN_AE_separate(300, combinations, 128, 32, 8, device, 8, 16, 6)
s = time.time()
loss = train_model(model, n_epochs, dataloader, train_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [13:08<00:00,  7.88s/it]


0.976 788.329


In [10]:
dataloader = create_mixed_batch_dataloader(dataset, 16)
model = create_TCN_AE_shared(300, 64, 16, 4, device, 8, 32, 6)
s = time.time()
loss = train_model(model, n_epochs, dataloader, train_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [13:14<00:00,  7.94s/it]


0.969 793.559


In [15]:
dataloader = create_mixed_batch_dataloader(dataset, 16)
model = create_LSTM_MAE(200, 300, combinations, device)
s = time.time()
loss = train_model(model, n_epochs, dataloader, train_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [7:58:07<00:00, 286.87s/it] 


0.945 28686.596


In [14]:
dataloader = create_dataloader(dataset, 64, 25)
model = create_LSTM_AE_separate(250, 300, combinations, device)
s = time.time()
loss = train_model(model, n_epochs, dataloader, eval_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [2:04:13<00:00, 74.53s/it]


0.825 7453.445


In [13]:
dataloader = create_mixed_batch_dataloader(dataset, 16)
model = create_LSTM_AE_shared(250, 300, device)
s = time.time()
loss = train_model(model, n_epochs, dataloader, eval_loss_function, optimizer, device, 
                            300, 0.0001, convergence_plot=False, return_training_loss=True).item()
e = time.time()
t = e-s
result, (fprs, tprs) = evaluate_on_dataset(model, test_data, eval_loss_function, device, show_roc_curve = False, shared_threshold = False)
print(round(result,3), round(t,3))

100%|██████████| 100/100 [2:09:05<00:00, 77.43s/it]


0.89 7743.215


In [11]:
train_data = torch.Tensor(dataset.df_torque.to_numpy()).unsqueeze(1).unsqueeze(1)
test_data_tensor = torch.Tensor(test_data.to_numpy()).unsqueeze(1).unsqueeze(1)
test_data_squeezed = test_data_tensor.squeeze()
model = create_DMOC(n_tasks, objective="hard-boundary", dimensions=32)
s = time.time()
net, loss, c_list = model.train(train_data, 100)
e = time.time()
t = e-s
eval_results = model.test(test_data_tensor)
fpr,tpr,thresholds = metrics.roc_curve(test_data_squeezed[:,-1].cpu(), eval_results)
result = metrics.auc(fpr,tpr)
print(round(result,3), round(t,3))

/home/snagelkerk/.local/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
100%|██████████| 4/4 [00:00<00:00, 14.36it/s]


  Epoch 1/100	 Time: 0.281	 Loss: 146656.00976562


100%|██████████| 4/4 [00:00<00:00, 19.71it/s]


  Epoch 2/100	 Time: 0.204	 Loss: 23186.43359375


100%|██████████| 4/4 [00:00<00:00, 19.43it/s]


...

  Epoch 98/100	 Time: 0.234	 Loss: 204.87121964


100%|██████████| 4/4 [00:00<00:00, 19.70it/s]


  Epoch 99/100	 Time: 0.205	 Loss: 170.00985336


100%|██████████| 4/4 [00:00<00:00, 19.71it/s]


  Epoch 100/100	 Time: 0.204	 Loss: 203.53484726
0.909 280.11
